## Prepare RAFS API Client

In [ ]:
import json
import os
import sys
from pprint import pprint
from utils import print_json

from dotenv import dotenv_values

sys.path.append("..")

from client.api.settings import ApiClientSettings 
from client.api_client import APIClient
from utils import print_json, load_wpc_payload

conf = dotenv_values("./demo.env")

api_client_dev = APIClient(
    conf["osdu_host"],
    "dev",
    conf["url_prefix"],
    conf["data_partition_id"],
    conf["access_token"],
)

## Dev Search Endpoints

/api/rafs-ddms/dev/samplesanalysis/{analysis_type}/search

/api/rafs-ddms/dev/samplesanalysis/{analysis_type}/search/data



### Search all RoutineCoreAnalysis, MultiStageSeparator, and NMR tests

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

analysis_types = ["routinecoreanalysis", "multistageseparator", "nmr"]

for analysis_type in analysis_types:
    search_response = api_client_dev.get(f"/samplesanalysis/{analysis_type}/search", headers=headers)
    print_json(search_response.json(), title=analysis_type)

### Search NMR tests data

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}
params = {"offset": 0, "page_limit": 100}

search_response = api_client_dev.get("/samplesanalysis/nmr/search/data", headers=headers, params=params)

print_json(search_response.json())

### Search MultiStageSeparator tests data

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}
params = {"offset": 100, "page_limit": 2}

search_response = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response.json())

## Filters & Aggregation

In [ ]:
# Formats
columsn_filter = ["ColumnNameA", "ColumnNameB"]
columns_aggregation = ["PropertyName", "aggregation_function"]
rows_filter = {"PropertyName": {"operator": "value"}}
rows_multiple_filter = {"logical-operator": [{"PropertyName": {"operator": "value"}}]}

# Comparison Operators
["$lt", "$gt", "$lte", "$gte", "$eq", "$neq"]

# Logical Operators
["$and", "$or"]

# Aggregation functions
["mean", "count", "max", "min", "sum", "describe"]

print()

### Aggregation on NMR data

In [ ]:
nmr_columns_aggregation_1 = '["NMRTest.NMRTestSteps.CumulativeWaterSaturation.Value", "sum"]'
nmr_columns_aggregation_2 = '["NMRTest.NMRSummaryData.Permeability.Value", "mean"]'

headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

params = {"columns_aggregation": nmr_columns_aggregation_1}
search_response_1 = api_client_dev.get("/samplesanalysis/nmr/search/data", headers=headers, params=params)

print_json(search_response_1.json())

params = {"columns_aggregation": nmr_columns_aggregation_2}
search_response_1 = api_client_dev.get("/samplesanalysis/nmr/search/data", headers=headers, params=params)

print_json(search_response_1.json())

### Rows Filter on NMR data

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_filter_nmr_1 = '''
{
    "NMRTest.NMRSummaryData.EchoSpacing.Value": {"$neq": 0.2}
}
'''
params = {"rows_filter": rows_filter_nmr_1}

search_response = api_client_dev.get("/samplesanalysis/nmr/search/data", headers=headers, params=params)

print_json(search_response.json())

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_filter_nmr_2 = '''
{
    "NMRTest.NMRSummaryData.Permeability.Value": {"$eq": 4740}
}
'''
params = {"rows_filter": rows_filter_nmr_2}

search_response = api_client_dev.get("/samplesanalysis/nmr/search/data", headers=headers, params=params)

print_json(search_response.json())

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_multiple_filter_nmr = '''
{
    "$and": [
        {
            "$or": [
                {
                    "$or": [
                        {
                            "SamplesAnalysisID": {"$eq": "opendes:work-product-component--SamplesAnalysis:469fdbf6482b4f3fab6eb9400d982fb4:"}
                        }
                    ]
                }, 
                {
                    "SamplesAnalysisID": {"$eq": "opendes:work-product-component--SamplesAnalysis:55dce95b591346b39c2c684841ba9705:"}
                }
            ]
        }, 
        {
            "SamplesAnalysisID": {"$eq": "opendes:work-product-component--SamplesAnalysis:469fdbf6482b4f3fab6eb9400d982fb4:"}
        }, 
        {
            "NMRTest.NMRTestSteps.T2FullySaturated.Value": {"$eq": 1259.0}
        }
    ]
}
'''
params = {"rows_multiple_filter": rows_multiple_filter_nmr}

search_response = api_client_dev.get("/samplesanalysis/nmr/search/data", headers=headers, params=params)

print_json(search_response.json())

### Aggregation on MSS data

In [ ]:
mss_columns_aggregation_1 = '["MultiStageSeparatorTestData.GasFormationVolumeFactor.Value", "describe"]'
mss_columns_aggregation_2 = '["CalculatedProperties.LiquidCalculatedProperties.FlashedLiquidDensity.Value", "sum"]'

headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

params = {"columns_aggregation": mss_columns_aggregation_1}
search_response_1 = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response_1.json())

params = {"columns_aggregation": mss_columns_aggregation_2}
search_response_1 = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response_1.json())

### Rows Filters on MSS data

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_filter_mss_1 = '''
{
    "CalculatedProperties.LiquidCalculatedProperties.FlashedLiquidDensity.Value": {"$neq": 5}
}
'''
params = {"rows_filter": rows_filter_mss_1}

search_response = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response.json())

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_filter_mss_2 = '''
{
    "LiquidCompositionalAnalysis.FlashedGasRelativeMass.Value": {"$eq": 0.8}
}
'''
params = {"rows_filter": rows_filter_mss_2}

search_response = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response.json())

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_multiple_filter_mss ='''
{
    "$and": [
        {
            "SaturationPressure.Value": {"$gt": 950}
        }, 
        {
            "ReservoirPressure.Value": {"$lt": 500}
        }
    ]
}
'''
params = {"rows_multiple_filter": rows_multiple_filter_mss}

search_response = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response.json())

In [ ]:
headers = {"Accept": "*/*;version=1.0.0", "Content-Type": "application/json"}

rows_multiple_filter_mss ='''
{
    "$and": [
        {
            "SaturationPressure.Value": {"$gt": 950}
        }, 
        {
            "ReservoirPressure.Value": {"$lt": 500}
        }
    ]
}
'''
mss_columns_aggregation_2 = '["CalculatedProperties.LiquidCalculatedProperties.FlashedLiquidDensity.Value", "sum"]'
params = {"rows_multiple_filter": rows_multiple_filter_mss, "columns_aggregation": mss_columns_aggregation_2}

search_response = api_client_dev.get("/samplesanalysis/multistageseparator/search/data", headers=headers, params=params)

print_json(search_response.json())